# Exercise 1

Produce a “people” file with the following schema. Save it as a CSV with a header line to the working directory.

In [1]:
import pandas as pd
from datetime import datetime
from typing import Dict, List, Any

Here's a utility function to load columns as specified datatypes.

In [2]:
def load_csv(filename: str,
             col_dtypes: Dict[str,Any],
             date_cols: List[str] = [],
             index_col: str = None
            ) -> pd.DataFrame:
    """ Function loads specific columns from csv files as specific datatypes into a DataFrame
    
    Parameters:
    filename (str): Path of file to load
    col_dtypes (dict): Map of column name to the dtype we require in the output df
    date_cols (list): List of column names that should be converted to datetime objects
    index_col (str): Optional column name or index to use as the DataFrame row index
    
    Returns:
    pandas.DataFrame
    
    Note: For timestamp types, we should specify str as the dtype in the col_dtypes 
    argument. The col_dtypes dict is passed directly to pandas read_csv() and datetime
    conversion is performed as a second step after reading data in from file. 
    
    The only date format allowed for columns listed in date_cols is as follows:

        Example: 
            Mon, 1979-03-05 21:08:54

        Format String:
            %a, %Y-%m-%d %H:%M:%S
        
        See https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
        for more information on strftime() format string conventions.
        
        Also see https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html for
        details on the utility function used to convert date strings to datetime objects
    """
    
    # Read csv file - only load columns specified in col_dtypes_keys
    df = pd.read_csv(filename, usecols=col_dtypes.keys(), dtype=col_dtypes)
    
    date_fmt = "%a, %Y-%m-%d %H:%M:%S"

    for col in date_cols:
        df[col] = pd.to_datetime(df[col], format=date_fmt)
    
    # Set the index column if provided
    if index_col:
        df = df.set_index(index_col)
        
    return df

### Constinuent Information

In [3]:
# cons.csv columns and data types to read
cons_dtypes = {
    'cons_id': int,
    'source': str, # note - storing as a string here, but might be more efficient as category
    'create_dt': str,
    'modified_dt': str
}

# load as a DataFrame
df_cons = load_csv('data/cons.csv',
                   col_dtypes = cons_dtypes,
                   date_cols = ['create_dt', 'modified_dt'],
                   index_col = 'cons_id'
                  )

# change NaN values back to blank strings in the source
df_cons.source = df_cons.source.fillna('')

df_cons.shape[0]

700000

In [4]:
# Now, let's examine the header line and first 3 rows of data
# df_cons.head(3)

In [5]:
# # Create a styler object and left align the text, then view header line and first 5 rows
# df_info_styled_hd = df_info.head().style
# df_info_styled_hd.set_properties(**{'text-align': 'left'})
# df_info_styled_hd.set_table_styles([{
#     'selector': 'th:not(.index_name)',
#     'props': 'text-align: left;'
# }])

In [6]:
# # Let's check the size of the data file and the data types for each of its columns
# df_info.info()

### Constinuent Email Addresses

***Note:***
* Boolean columns (including 'is_primary') in all of these datasets are 1/0 numeric values. 1 means True, 0 means False. (We only want primary email addresses)

In [7]:
# cons_email.csv columns and data types to read
email_dtypes = {
    'cons_email_id': int,
    'cons_id': int,
    'email': str,
    'is_primary': bool # casting these as booleans, so output will be True/False rather than 0/1  
}

# load file as a DataFrame
df_email = load_csv('data/cons_email.csv',
                    col_dtypes = email_dtypes,
                    index_col = 'cons_email_id'
                   )
# We only want to include primary emails (see schema for people file)
# so filter to include only rows where values of is_primary is True 
df_email = df_email.loc[df_email['is_primary']]


# We want to make sure each row has a unique email
rows = df_email.shape[0] # count number of rows 
unique_emails = len(df_email.email.unique()) # count number of unique values in email column
assert rows == unique_emails # if these are equal, there are no duplicate emails (otherwise will throw error)
rows

605639

In [8]:
# View first 3 rows
# df_email.head(3)

In [9]:
# # View last 3 rows
# df_email.tail(3)

In [10]:
# Check the size of the data file and data types for each column
# df_email.info()

### Constinuent Subscription Status

***Note:***
* We only care about subscription statuses where chapter_id is 1.
* If an email is not present in this table, it is assumed to still be subscribed where chapter_id is 1. 

In [11]:
# cons_email_chapter_subscription.csv columns and data types to read
subs_dtypes = {
    'cons_email_chapter_subscription_id': int,
    'cons_email_id': int,
    'chapter_id': int,
    'isunsub': bool
}

# load file as a DataFrame
df_subs = load_csv('data/cons_email_chapter_subscription.csv',
                   col_dtypes = subs_dtypes,
                   index_col = 'cons_email_chapter_subscription_id'
                  )

# "We only care about subscription statuses where chapter_id is 1", so filter to 
# include only rows where chapter_id == 1
df_subs = df_subs.loc[ df_subs.chapter_id == 1 ]
df_subs.shape[0]

275484

In [12]:
# 431,649 email addresses have no subscriptions! (assume subscribed to chapter_id 1)
# From instructions:
# "If an email is not present in this table, it is assumed to still be subscribed where chapter_id is 1"
# So, we need to know which email addresses in df_email do not have a subscription record in df_subs
# To find this out, we do an anti-join between df_email and df_subs, then save as a new DataFrame
df_sub_assumed = df_email.merge(df_subs, on='cons_email_id', how='outer', indicator=True) # add col called _merge containing str indicating which df record came from
df_sub_assumed = df_sub_assumed[df_sub_assumed._merge == 'left_only'].drop('_merge', axis=1) # now drop _merge column, we don't need it any more
df_sub_assumed = df_sub_assumed.set_index('cons_email_id') # setting index here for neatness
df_sub_assumed = df_sub_assumed[['email']] # now limit columns to include only cons_email_id and email
df_sub_assumed['isunsub'] = False # now add col called isunsub and set to False (because we're making the assumption these are all subscribed chapter 1)

# Now let's ensure we have no duplicate emails in df_nosub
rows = df_sub_assumed.shape[0] # check number of rows in df_nosub
unique = len(df_sub_assumed.email.unique())
assert rows == unique

rows

431649

In [13]:
# Now create a new df containing every email from df_email that does have a record in df_subs
df_sub_confirmed = df_email.merge(df_subs, on='cons_email_id', how='inner')
df_sub_confirmed = df_sub_confirmed.set_index('cons_email_id')

# We only want to include cons_email_id, cons_id, email, and isunsub cols, so drop the others
df_sub_confirmed = df_sub_confirmed.drop(['is_primary', 'chapter_id'], axis=1)

# Check that every email is unique
rows = df_sub_confirmed.shape[0]
unique = len(df_sub_confirmed.email.unique())
assert rows == unique
rows

173990

In [14]:
# Concatentate df_sub_assumed with df_sub_confirmed 
df_email_comb = pd.concat([df_sub_confirmed, df_sub_assumed])

# Check that we have exactly one unique record for every primary email
rows = df_email_comb.shape[0]
assert rows == unique_emails
assert rows == len(df_email.email.unique())
rows

605639

### People

In [15]:
# Prepare "people" table
df_ppl = df_email_comb.join(df_cons, on='cons_id')
df_ppl = df_ppl.reset_index()
df_ppl = df_ppl.drop(['cons_id', 'cons_email_id'], axis = 1)

In [16]:
# now rename cols: source->code, isunsub->is_unsub, create_dt->created_dt, and modified_dt->updated_dt
df_ppl.rename(columns={'source':'code', 'isunsub':'is_unsub', 'create_dt':'created_dt', 'modified_dt': 'updated_dt'}, inplace=True)
df_ppl

,email,is_unsub,code,created_dt,updated_dt
0,aaron64@yahoo.com,True,,1992-06-01 06:07:45,1986-07-28 03:41:12
1,wyattvincent@hotmail.com,True,,1993-05-23 08:00:18,1983-05-07 09:29:18
2,tspencer@hotmail.com,True,twitter,1986-10-31 03:24:05,1979-09-22 05:01:01
3,ogarcia@gmail.com,True,,2010-10-03 05:49:12,1996-02-03 15:32:15
4,madeline69@mccarthy-jackson.com,True,twitter,2007-08-22 02:46:01,2014-07-15 01:05:24
...,...,...,...,...,...
605634,smallmelvin@mitchell.com,False,NaN,NaT,NaT
605635,gardnerchristian@hotmail.com,False,NaN,NaT,NaT
605636,ginanguyen@munoz.com,False,NaN,NaT,NaT
605637,tatenicole@yahoo.com,False,NaN,NaT,NaT
